In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType,DateType,FloatType
from pyspark.sql.functions import col ,lit,concat,to_timestamp,concat_ws

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/Common_functions"

In [0]:
# Create a text widget in Databricks with a default value "2021-03-21"
dbutils.widgets.text("p_file_date","2021-03-21")

# Retrieve the value entered in the widget and store it in a variable
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
##### Step 1 - Read the JSON file using the spark dataframe reader

pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])
pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.option("multiLine", True)\
.json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")


In [0]:
# Step 2 - Rename columns and add new columns
#1. Add ingestion_date with current timestamp
pit_stops_with_ingestion_date_df = add_ingestion_date(pit_stops_df)

#  2. Rename driverId and raceId
final_df = pit_stops_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("ingestion_date", current_timestamp()) \
.withColumn("file_date", lit(v_file_date))

In [0]:
# Step 3 - Write to output to processed container in Delta Lake

merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.stop = src.stop AND tgt.race_id = src.race_id"
merge_delta_data(final_df, 'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
%sql
SELECT race_id, count(1) FROM f1_processed.pit_stops GROUP BY race_id

race_id,count(1)
1053,56
1052,40
